In [62]:
import FinanceDataReader as fdr
import numpy as np

etfkr = fdr.StockListing("ETF/KR")
krx = fdr.StockListing("KRX")

def search_code(name):
    code = krx[krx['Name'] == name]['Code']
    # 결과가 있으면 해당 종목 코드를 반환하고, 없으면 None 반환
    if not code.empty:
        return code.values[0]

    code = etfkr[etfkr['Name'].str.strip() == name]['Symbol']
    # display(code)    
    if not code.empty:
        return code.values[0]
        
    return np.nan

종목명 = 'RISE 미국30년국채엔화노출(합성 H)'
코드 = search_code(종목명)
display(코드)
'RISE 미국채30년엔화노출(합성 H)'
# test = etfkr[etfkr['Name'].str.contains('RISE')]
# display(test['Name'].values)
# test['Name'].values == 'RISE 미국30년국채엔화노출(합성 H)'


'472870'

In [49]:
import pandas as pd

# CSV 파일을 줄 단위로 읽기
file_path = '6401254822-20240907.csv'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 홀수 줄과 짝수 줄을 결합하여 하나의 줄로 만듬
merged_lines = []
for i in range(0, len(lines) - 1, 2):
    merged_line = lines[i].strip() + ',' + lines[i + 1].strip()  # 쉼표를 이용해 두 줄 결합
    merged_lines.append(merged_line)

# 결합된 데이터를 DataFrame으로 바로 변환
from io import StringIO

# 결합된 라인들을 하나의 문자열로 변환
merged_data = "\n".join(merged_lines)

# pandas에서 이를 읽어들이기 위해 StringIO 객체로 변환
df = pd.read_csv(StringIO(merged_data))

# 결과 출력
# display(df)

# df['거래금액']을 숫자로 변환
df['거래금액'] = df['거래금액'].str.replace(',', '').astype(int)
df['잔액'] = df['잔액'].str.replace(',', '').astype(int)
df['정산금액'] = df['정산금액'].str.replace(',', '').astype(int)

df['종목명'] = df['종목명'].apply(lambda x: x.replace('KBSTAR', 'RISE') if pd.notnull(x) and 'KBSTAR' in x else x)
df['종목'] = df['종목명'].apply(search_code)

df2 = df[['거래일', '거래종류', '거래금액', '수수료', '세금', '정산금액', '잔액', '거래단가', '거래수량', '유가잔고', '종목명', '종목']]
display(df2)

입금 = df2[df2['거래종류'].str.contains('입금|예탁금이용료')]
출금 = df2[df2['거래종류'].str.contains('출금')]
display(입금)
display(출금)
총입금 = 입금['정산금액'].sum() 
총출금 = 출금['정산금액'].sum()
display(총입금)
display(총출금)
총수익 = 총출금 - 총입금
총수익

,거래일,거래종류,거래금액,수수료,세금,정산금액,잔액,거래단가,거래수량,유가잔고,종목명,종목
0,2023.12.22,Smart+당사이체입금,5000000,0,0,5000000,5000000,0,0,0,NaN,NaN
1,2023.12.27,Smart+당사이체입금,1000000,0,0,1000000,6000000,0,0,0,NaN,NaN
2,2024.1.2,Smart+거래소주식매수,5266000,221,0,5266221,733779,"105,320",50,50,KODEX KOFR금리액티브(합성),423160
3,2024.1.2,Smart+거래소주식매수,98950,4,0,98954,634825,"9,895",10,10,RISE 미국채30년엔화노출(합성 H),NaN
4,2024.1.3,Smart+거래소주식매수,100600,4,0,100604,534221,"10,060",10,20,RISE 미국채30년엔화노출(합성 H),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
249,2024.9.5,Smart+거래소주식매수,1082900,45,0,743669,-339276,"15,470",70,70,ACE KRX금현물,411060
250,2024.9.5,Smart+거래소주식매수,89730,3,0,0,-429009,"9,970",9,300,SOL 미국30년국채커버드콜(합성),473330
251,2024.9.5,Smart+거래소주식매수,42020,1,0,0,-471030,"10,505",4,4,RISE 글로벌리얼티인컴,475380
252,2024.9.5,Smart+거래소주식매도,466120,19,0,0,-4929,"10,840",43,100,ACE 미국달러SOFR금리(합성),456880


,거래일,거래종류,거래금액,수수료,세금,정산금액,잔액,거래단가,거래수량,유가잔고,종목명,종목
0,2023.12.22,Smart+당사이체입금,5000000,0,0,5000000,5000000,0,0,0,NaN,NaN
1,2023.12.27,Smart+당사이체입금,1000000,0,0,1000000,6000000,0,0,0,NaN,NaN
37,2024.2.2,ETF분배금입금,1300,0,0,1300,474745,0,0,0,한국투자 ACE 미국S&P500증권상장지수투자신탁(주식),NaN
38,2024.2.2,ETF분배금입금,35,0,0,35,474780,0,0,0,삼성 KODEX 미국반도체MV증권상장지수투자신탁[주식],NaN
39,2024.2.2,ETF분배금입금,15,0,0,15,474795,0,0,0,KB RISE 미국반도체NYSE증권상장지수투자신탁(주식)(H),NaN
51,2024.2.14,타사이체입금,1000000,0,0,1000000,1003586,0,0,0,NaN,NaN
59,2024.2.20,타사이체입금,1000000,0,0,1000000,1151506,0,0,0,NaN,NaN
64,2024.2.28,타사이체입금,1000000,0,0,1000000,1167017,0,0,0,NaN,NaN
67,2024.3.4,예탁금이용료,1560,0,0,1560,898617,0,0,0,NaN,NaN
108,2024.4.2,ETF분배금입금,176,0,0,176,1501634,0,0,0,한국투자 ACE 미국배당다우존스증권상장지수투자신탁(주식),NaN


,거래일,거래종류,거래금액,수수료,세금,정산금액,잔액,거래단가,거래수량,유가잔고,종목명,종목


12113331

0

-12113331

In [33]:
import pandas as pd
import numpy as np

# 예시 DataFrame 생성 (NaN 값 포함)
data = {'Name': ['KBSTAR 코스피', '삼성전자', np.nan, 'KBSTAR 채권', 'LG전자']}
df = pd.DataFrame(data)

# 'Name' 컬럼에서 NaN이 아닌 값 중 'KBSTAR'가 포함된 경우 'RISE'로 변경
df['Name'] = df['Name'].apply(lambda x: x.replace('KBSTAR', 'RISE') if pd.notnull(x) and 'KBSTAR' in x else x)

# 결과 출력
print(df)


       Name
0  RISE 코스피
1      삼성전자
2       NaN
3   RISE 채권
4      LG전자
